In [1]:
import os
os.chdir('..')

In [2]:
from team_functions import *

In [3]:
today = date.today()
print(f'Today is: {today}')

Today is: 2024-02-06


In [4]:
last_games = pd.read_excel('Predictions/games_odds.xlsx', 'Tonight')
prev_games = pd.read_excel('Predictions/games_odds.xlsx', 'Previous')

In [5]:
options = Options()
options.add_argument('--headless')
driver = Firefox(options=options)

In [6]:
url = 'https://www.basketball-reference.com/boxscores/'
soup = getSoup(url, driver=driver)

In [7]:
summaries = soup.find('div', {'class': 'game_summaries'})
games = summaries.findAll('table', {'class': 'teams'})

games_df = []
for idx, game in enumerate(games):
    loser = game.find('tr', {'class': 'loser'}).find('td').find('a').get('href').split('/')[2]
    winner = game.find('tr', {'class': 'winner'}).find('td').find('a').get('href').split('/')[2]
    
    game_row = {'Winner': winner, 'Loser': loser}
    games_df.append(game_row)
    
games_df = pd.DataFrame(games_df)

In [9]:
results = []
for team in last_games['visitor']:
    if team in list(games_df['Winner']):
        results.append(1)
    elif team in list(games_df['Loser']):
        results.append(0)
    else:
        print(f'Error checking results of team: {team}')

last_games['Results'] = np.array(results)

Error checking results of team: ORL
Error checking results of team: PHO


In [10]:
last_games

,date,visitor,local,v_line,l_line,Results
0,2024-02-04,ORL,DET,1.359712,3.25,1
1,2024-02-04,PHO,WAS,1.160000,5.55,1


In [11]:
prev_games

,date,visitor,local,v_line,l_line,Results
0,2024-02-03,BRK,PHI,2.420000,1.588235,1
1,2024-02-03,GSW,ATL,2.200000,1.704225,0
2,2024-02-03,SAC,CHI,2.050000,1.800000,1
3,2024-02-03,LAL,NYK,2.640000,1.505051,1
4,2024-02-03,MIL,DAL,1.540541,2.540000,1
5,2024-02-03,CLE,SAS,1.219780,4.500000,1


In [12]:
prev_games = pd.concat([prev_games, last_games])

In [13]:
url = 'https://sportsbook.draftkings.com/leagues/basketball/nba'

In [14]:
driver.get(url)

html = driver.page_source
soup = bs(html, 'lxml')

driver.close()

In [15]:
def american_to_decimal(american):
    '''
    This functions transform an american system odd (e.g. -120) to a decimal system (e.g 1.8)
    
    INPUTS:
    player: row containing player stats
    
    OUTPUT:
    new_player: copy of the player df input with the updated Odd value in Decimal System
    '''
    # Check if the value is positive or negative (e.g. Odd = +100 or Odd = -120)
    # Transform it to decimal with the given formula
    if american > 0:
        decimal = (american/100) + 1
    else:
        decimal = (100/abs(american)) + 1
    
    return decimal

In [16]:
table = soup.find('tbody', {'class': 'sportsbook-table__body'})
rows = table.findAll('tr')
games, game, idx = [], {'date': today, 'visitor': '', 'local': '', 'v_line': 0, 'l_line': 0}, 0
for row in rows:
    team_name = row.find('div', {'class': 'event-cell__name-text'}).text.split(' ')
    if len(team_name[0]) == 3:
        team_id = team_name[0]
    else:
        team_id = team_name[0] + team_name[1][0]
    team_id = team_id.replace('BKN', 'BRK')
    odd = row.find('span', {'class': 'sportsbook-odds american no-margin default-color'}).text.replace('−', '-')
    
    if idx == 0:
        game['visitor'] = team_id
        game['v_line'] = american_to_decimal(int(odd))
        idx = 1
        
    elif idx == 1:
        game['local'] = team_id
        game['l_line'] = american_to_decimal(int(odd))
        idx = 0
        
        games.append(game)
        game = {'date': today, 'visitor': '', 'local': '', 'v_line': 0, 'l_line': 0}
    
games_df = pd.DataFrame(games)
games_df

,date,visitor,local,v_line,l_line
0,2024-02-06,HOU,IND,3.750000,1.289855
1,2024-02-06,MEM,NYK,8.000000,1.090909
2,2024-02-06,ORL,MIA,2.300000,1.645161
3,2024-02-06,DAL,BRK,1.645161,2.300000
4,2024-02-06,MIN,CHI,1.487805,2.700000
5,2024-02-06,OKC,UTA,1.645161,2.300000
6,2024-02-06,MIL,PHO,2.400000,1.602410


In [17]:
with pd.ExcelWriter('Predictions/games_odds.xlsx') as writer:  
    games_df.to_excel(writer, sheet_name='Tonight', index=0)
    prev_games.to_excel(writer, sheet_name='Previous', index=0)